# Portfolio Math  and Diversification

---

**🎯 Learning Objectives**

In this section we will learn how to do some basic math at portfolio level. You will know how to calculate return, expected return, and volatility given portfolio weights.

By the end of this chapter, you should be able to:


1. **See risk through a portfolio lens.**  
   Grasp Markowitz’s insight that an asset’s danger depends on what it adds to the *whole* mix, not on its stand-alone volatility.

2. **Express returns, means, and variance with matrix algebra.**  
   Use $R_p   =  R W$, $E[R_p]=W' E[R]$, and $\text{Var}(R_p)=W' \Sigma W$ to avoid loops and scale instantly from two assets to hundreds.

3. **Quantify diversification benefits.**  
   Investigate how adding a higher-volatility asset can *lower* portfolio variance when correlations are below one, and plot the US–international efficient frontier.

4. **Compute and interpret portfolio weights.**  
   Distinguish long-only, short, and leveraged positions; verify that weights sum to one (or note where leverage breaks that rule).

---


## Table of Contents

1. Libraries and Auxiliary Functions
2. Why do you need to think about portfolios?
3. Portfolio weights
4. Our Data set
5. Accessing Data in DataFrames
6. Portfolio Returns
7. Portfolio Expected returns
8. Portfolio Variance
9. Checking that Linear Algebra works
10. Diversification
11. Tracing the Investment Frontier
12. Variance and portfolio weights
13. The Mean-Variance Frontier

## Libraries and Auxiliary Functions

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## Why do you need to think about portfolios?


1. The great insight from Harry Markowitz was to think of risk in terms of what the stock adds to your portfolio

1. Just like sugar can be good for you if you are not eating any sugar, it is terrible if you are eating a lot of it

1. What investors should care about is, just like eaters, their final diet. If a given stock brings a lot of what you already have, it will be bad for you, i.e., **risky**.


1. So volatility is a good gauge of risk at the portfolio level, because it is asking how your whole portfolio behaves, which you obviously should care about. If it goes down a lot, it means you cannot buy stuff!

1. But when thinking about a specific stock, it's volatility means very little.

1. Unless your entire portfolio is just that stock, you don't really need to bear the stock risk--if you have 1% in a stock and the stock drops to ZERO, that is only 1% in your portfolio, so at most you lose 1%. So whatever volatility this stock might have you SHOULD not perceive the stock as very risky for you.

1. As long, of course, your position in the stock remains small. As you buy more,  It will **becomes** risky **for you** because the more you have, the more your portfolio moves like it

1. Note that if this stock would move together with other pieces of your portfolio then your calculation should be very different. If this stock goes to zero exactly when your other assets are also losing a lot of money, this will feel very risky trade.

1. The amount of covariance across stocks will be a key determinant of how much we can hold very volatile stocks without adding much overall risk to our portfolio

## **Portfolio weights**

* The portfolio weight for stock $j$ , denoted $w_j$, is the fraction of a portfolio value held in stock $j$

  $$w_j=\frac{\text{Dollar held in stock j}}{\text{Dollar value of portfolio}}$$
  
* By construction, the portfolio weights allways add up to one: you invest all you got somewhere, and nothing more

  * This doesn't mean that you can't borrow to invest, just means that you will have a negative weight somewhere offsetting the positive position in the other assets
  * But if the weights do not add up to one, then it is NOT a portfolio!

   $$\sum_{j=1}^N w_j=1$$
  
  * In matrix notation
  
  $$\mathbf{1}'W=1$$
  
  * where $\mathbf{1}$ is a N by 1 vector of 1's (i.e. a vector with entry 1 in each position) and $W$ is the N by 1 vector of portfolio weights

## Our Data set

> We will think about portfolios using a data set that contains monthly data on returns of
> - diversified portfolios of US  equities
> - XUS developed market equities (typically refered as "international" in the industry)
> - emerging market equities
> - US  government bonds
> - XUS developed markets government bonds
>
> We will work directly with excess returns as it is more convenient in this case

In [ ]:
# We will import a the data set from our github repository by pointing directly to the url

url="https://raw.githubusercontent.com/amoreira2/Lectures/main/assets/data/GlobalFinMonthly.csv"
# this is a csv file so I am using pandas read_csv function
Data = pd.read_csv(url)
# Here I tell python that the Date column is a date, to_datetime is a pandas function that converts strings to dates
Data['Date']=pd.to_datetime(Data['Date'])
#Here I set the Date column as the index of the dataframe. This makes it easier to work with time series data as I can use date based indexing
Data=Data.set_index(['Date'])

# make it excess returns by subtracting the risk-free rate
Rf=Data['RF']
Data=Data.drop(columns=['RF']).subtract(Data['RF'],axis=0)
Data
# what do you notice in the data below?

,MKT,USA30yearGovBond,EmergingMarkets,WorldxUSA,WorldxUSAGovBond
Date,,,,,
1963-02-28,-0.0238,-0.004178,0.095922,-0.005073,NaN
1963-03-31,0.0308,0.001042,0.011849,-0.001929,-0.000387
1963-04-30,0.0451,-0.004343,-0.149555,-0.005836,0.005502
1963-05-31,0.0176,-0.004207,-0.014572,-0.002586,0.002289
1963-06-30,-0.0200,-0.000634,-0.057999,-0.013460,0.000839
...,...,...,...,...,...
2016-08-31,0.0050,-0.008617,0.024986,0.000638,-0.009752
2016-09-30,0.0025,-0.016617,0.012953,0.012536,0.009779
2016-10-31,-0.0202,-0.049660,0.002274,-0.020583,-0.043676


In [ ]:
# drop rows with missing values
Data=Data.dropna()
Data

,MKT,USA30yearGovBond,EmergingMarkets,WorldxUSA,WorldxUSAGovBond
Date,,,,,
1963-03-31,0.0308,0.001042,0.011849,-0.001929,-0.000387
1963-04-30,0.0451,-0.004343,-0.149555,-0.005836,0.005502
1963-05-31,0.0176,-0.004207,-0.014572,-0.002586,0.002289
1963-06-30,-0.0200,-0.000634,-0.057999,-0.013460,0.000839
1963-07-31,-0.0039,0.000700,0.085891,0.005200,-0.000799
...,...,...,...,...,...
2016-08-31,0.0050,-0.008617,0.024986,0.000638,-0.009752
2016-09-30,0.0025,-0.016617,0.012953,0.012536,0.009779
2016-10-31,-0.0202,-0.049660,0.002274,-0.020583,-0.043676


## Accessing Data in DataFrames

In [ ]:
# for example here is the return on a particular month
display(Data.loc['2008-09'])
# a more clear way of accessing a particular month
display(Data.loc['2008-09',:])
# acccessing a particular asset
display(Data['SP500'])
# a more clear way of accessing a particular asset
display(Data.loc[:,'EmergingMarkets'])
# accessing a particular asset in a particular month
display(Data.loc['2008-09','MKT'])
# acessing multiple assets (say two assets) in a particular month?

# acessing 1990 to 2000 for all assets


## Portfolio Returns

**Portfolio returns** are just the dollar-weighted average of the individual position returns

$$ r_p=\sum_{j=1}^N w_jr_j=W'R$$


* Where $R$ is the N by 1 vector of realized asset returns

* I use big R and big W here to emphasize that they are vectors, like  ($[r^{RF},r^{MKT},..]$), ($[w^{RF},w^{MKT},..]$),

* I use litlle $r_p$ becasue the return on a portfolio is just a scalar




In [ ]:
# lets start by constructing an equal-weighted portfolio
W=np.ones((5,1))/5

display(W)
# it adds up to 1
display(np.sum(W))
# It has dimensions
display(W.shape)




array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

### ⚡ ⏰ Question: G4 ⚡

- How we construct the returns of this portfolio for a particular month? Say 9/2008?
- What do I need to pay attention to? I want the matrix way of constructing portfolio returns
- Show me the code

### ⚡ ⏰ Question: G4 (yes,still you)⚡

- Now lets say we want to compute the returns of your portfolio across all dates
- Do we need to do a for loop?
- NO! Avoid a for loop like the plague! Matrix algebra-- believe it or not not--is your friend
- How do you obtain the dataframe below with returns for our portfolio in each single date?
_____________________________________
**⛅ A hint**

- R is a vector T by N of historical asset returns (T periods N assets)

- W is a vector N by 1 of weights

- Then how do I get something that is T by 1 that does what I want

- Each row of R has the date t returns. When we do this "vector" multiplication we multiply each rwo of the matrix on the left with the vector on the right, so each row of the resulting vector is exactly the returns of our portfolio in a given date!


_____________________________________


In [ ]:
# complete the __
Rp=__
display(Rp)
Rp.plot()

,0
Date,
1963-03-31,0.008275
1963-04-30,-0.021826
1963-05-31,-0.000295
1963-06-30,-0.018251
1963-07-31,0.017418
...,...
2016-08-31,0.002451
2016-09-30,0.004230
2016-10-31,-0.026369


## Portfolio Expected returns

* Averages and expectations are linear
   * If you expect tomorrow temperature to be 60F, how much you expect 4*"tomorrow temperature" to be?
     
* The expected return of a portfolio is the dollar weighted average of the expected returns of the individual assets.


$$E[R_p]=E[\sum_{j=1}^N w_jr_j]=\sum_{j=1}^N w_jE[r_j]=W'E[R]$$

This means that our average estiamtor also have this property!

$$\hat{E}[R_p]=\sum_t^T \frac{R_{p,t}}{T}=\sum_t^T \frac{\sum_{j=1}^N w_jr_{j,t}}{T}=\sum_{j=1}^N w_j\sum_t^T \frac{r_j}{T}=W'\hat{E}[R]$$


* Lets compute for our equal weighted portfolio:

In [ ]:
# estimate the expected excess return of each asset
E_hat=Data.mean()
display(E_hat)
# This is average excess return in the sample
# so if you pick a random month in the sample, this is the excess return you expect to get
display(E_hat*12)
# What is the fundamental assumption we are making here?

# Now from our estimate of asset risk-premiums we can estimate the risk-premium of our portfolio
E_rp=W.T @ E_hat
display(E_rp)
# or we can also do
E_rp=E_hat @ W
display(E_rp)
# they both work.
# Because the weights are CONSTANT we can also simply take the average of the portfolio returns
E_rp=Rp.mean()
display(E_rp)

# they are the same!


,0
MKT,0.005140
USA30yearGovBond,0.002523
EmergingMarkets,0.006923
WorldxUSA,0.004149
WorldxUSAGovBond,0.002054


## Portfolio Variance

Why the portfolio variance is not just the dollar weighted averages of the asset variances?


**Two asset case:**

$$Var(r_p)= Var(w_1r_1+w_2r_2)=Cov(w_1r_1+w_2r_2,w_1r_1+w_2r_2)$$

* where we used that $Var(x)=Cov(x,x)$

* This yields the classic formula

$$Var(r_p)= w_1^2Var(r_1)+2w_2w_1Cov(r_2,r_1)+w_2^2Var(r_2)$$

**N-asset case**

From the "term distribution"  above it is intuitive what the N asset case would look like


$$Var(r_p)= Var(\sum_{j=1}^N w_jr_j)=\sum_{j=1}^N \sum_{i=1}^N w_jw_iCov(r_j, r_i)$$


* For a portfolio of 50 assets, this expression has 50 variance terms and 2450 covariance terms!



* We can use our matrix algebra notation to again avoid a nasty double for loop

$$\sum_{j=1}^N \sum_{i=1}^N w_jw_iCov(r_j, r_i)=W' Cov(R) W$$

* where $Cov(R)$ is the N by N variance covariance matrix of the assets and W is the vector of weights

In [ ]:
# We need to estimate the covariance matrix of asset returns
Cov_hat=Data.cov()
display(Cov_hat)
print(W.shape)
print(Data.cov().shape)
# so you see that the vectors and matrixes are nice and CONFORMING
# Why? Becuase we have have weights assiged to each asset
# so even if we dont; want an asset in our portfolio we need the weight there
# it just needs to be zero
Var_rp=W.T @ Cov_hat @ W
Var_rp

,MKT,USA30yearGovBond,EmergingMarkets,WorldxUSA,WorldxUSAGovBond
MKT,0.001950,0.000111,0.001298,0.001265,0.000187
USA30yearGovBond,0.000111,0.001229,-0.000204,-0.000013,0.000264
EmergingMarkets,0.001298,-0.000204,0.003550,0.001664,0.000249
WorldxUSA,0.001265,-0.000013,0.001664,0.002185,0.000422
WorldxUSAGovBond,0.000187,0.000264,0.000249,0.000422,0.000407


### Checking that Linear Algebra works

We can check that this vector notation deliver the same as the double sum by using two for loops


$$\sum_{j=1}^N \sum_{i=1}^N w_jw_iCov(r_j, r_i)=W' Cov(R) W$$



In [ ]:
cov=Data.cov()
covariance_sum=0 #initiate the sum at zero
for i in cov.index: # loop across all assets
    for j in cov.columns:
        i_pos=cov.index.get_loc(i) # this gets the position of the particular asset so we can locate the proper posiiton on the vector
        j_pos=cov.columns.get_loc(j) # same thing, but for the other asset in the double sum
        covariance_sum=covariance_sum+cov.loc[i,j]*W[i_pos,0]*W[j_pos,0]


covariance_sum

np.float64(0.0007923455067886969)

### **⚡ ⏰ Question: G3 ⚡**

- Write code that adjusts the current portfolio to have zero weight in emerging markets and the rest equal weighted
- Compute this portfolio Sharpe ratio in the last 10 years of the sample

In [ ]:
W[_,_]=_
W=W/np.__(W)

,0
0,0.000792


## Diversification

* A key concept in investing is diversification

* The famous: "don't put all your eggs in one basket" advice

* There are potential benefits of diversification for an investor when there are assets that  are imperfectly correlated with the investor portfolio

* So lets look at this from the vantage point of a US investors that is fully invested in the US equity market portfolio and is considering the benefits of investing in other world equity markets (WorldxUSA)

* We are ignoring bonds and emerging markets for now

In [ ]:
# here is the co-movement across the asset
display(Data[['MKT','WorldxUSA']].corr())
# What is noteworthy about this correlation matrix?
# Are there any benefits of diversification?


# But  note that the US market is less volatile than the international market

# Which portfolio you expect to have the lowest vol given that information?

# Which portfolio achieves the lowest vol? x_us=1? x_us=0? something else?
Data[['MKT','WorldxUSA']].std()*12**0.5


,MKT,WorldxUSA
MKT,1.000000,0.613103
WorldxUSA,0.613103,1.000000


### How the variance of the  portfolio varies with it's weight on international stocks?



In [ ]:
# given a particular weight on the US market we can construct our portfolio weights
x_us=1
x_int=1-x_us
W=np.array([x_us,x_int])


Rp=Data.loc[:,['MKT','WorldxUSA']]@ W
display(Rp.std()*12**0.5)
# What if we put all the weight on the international market?
x_us=0
x_int=1-x_us
W=np.array([x_us,x_int])
Rp=Data.loc[:,['MKT','WorldxUSA']]@ W
display(Rp.std()*12**0.5)

0.15295861105099787

## Tracing the Investment Frontier

### Variance and portfolio weights

**⚡ ⏰ Question: G2 ⚡**

We want to construct a plot of how portfolio variance varies as function of the weight on the international market. How do I do this?

Hint:  construct a list with weights from zero to 1 with  a bunch of values in the middle

In [ ]:
## G2 people:  complete the code below so we have a nice plot of the weight on the international market vs the portfolio vol. Please report annualized numbers. What is the frequency of the data? As you complete each step, uncomment to work in the next steps

##Step1: create a vector w that goes from 0 to 1 by 0.01
w=np.__(_,_,_)
display(w)

## Once we have these weights we can construct our portfolio weights and compute the portfolio vol.
## we then want to do do that for each weight in w and save the results in a list


## Step 2:  compute the portfolio vol for each weight in w and save the results in a list

#D=Data.loc[:,['MKT','WorldxUSA']]
#UsW=[]


# for __ in __ :
#     W=np.array([x_us,1-x_us])
#     vol=(___*(_))**0.5
#     UsW.append([x_us,1-x_us,vol])
# UsW=np.array(UsW)

## Step3: Make a scatter plot

# plt.scatter(UsW[:,_],UsW[:,_])
# plt.xlabel('Weight on World')

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

### The Mean-Variance Frontier

* We can also look at the investment frontier that such an investor faces: How her expected returns change with the variance

* always keep in mind that this is only the **in-sample** investment frontier because we are relying on the sample averages to construct it

**⚡ ⏰ Question: G1 ⚡**

In [ ]:
# G1 people:  complete the code below so we have a nice plot of the portfolio variance (in the x-axis) and the portfolio expected excess return in the y-axis . Please report annualized numbers. What is the frequency of the data?
UsW=[]
for __ in __:
    W=np.array([x_us,1-x_us])

    var=(__)*(__)
    er=(__)*(__)

    # save the weight on the world market as the first element of UsW
    # save the annualized variance of the portfolio as the second element of UsW
    # save the annualized expected excess return of the portfolio as the third element of UsW
    UsW.append([1-x_us,var,er])
UsW=np.array(UsW)

plt.scatter(UsW[:,__],UsW[:,__])
plt.xlabel('Variance')
plt.ylabel('Average excess return')

### Before we go: ⚡ ⏰ Question: G1 - G4 ⚡

Please think about these questions-use your intuition and your knowledge of foundations

* Which allocation should you choose?

* How international  investors benefit from holding US stocks?

* How US investors benefit of holding international stocks?

* Which asset is assets is safer for a an investor that is fully invested in the US market? And for an investor fully invested in the international market?

* Assuming that those are the only 2 assets available:
    - What portfolio an extremely risk-averse investor will choose?
    - As the investor becomes less risk-averse how he will change his portfolio?
    - Would anyone ever invest 100% in the US market?  and 100% in the international market?

* What happens with your answers above if we introduce free borrowing and lending at the risk-free rate?
  


---

**📝 Key Takeaways**


- **Matrix algebra is your friend.**  A single line replaces nested loops when calculating returns, expectations, and variance.  
- **Linear Algebra** is much easier to use than you think
- **Diversification is all about correlations.**  Pairing volatile but imperfectly correlated assets can cut overall risk more than simply holding the lower-volatility asset alone.  
- **How Risky is an asset** fundamentally depends of the portfolio of who is asking





---

   